# Gun Violence Data Initial json build
#### This notebook is obsolete see main.py 

Load data for all gun related incidents first pass for all incidents in dataset
Sources: original data source gunviolencearchive.org the data used is the property of gunviolencearchive.org. 
Any furture use of any of the data used in this project requires citation. 
https://www.gunviolencearchive.org/

In [8]:
# Dependencies and Setup
import pandas as pd
# import datetime as dt
# import numpy as np
from sqlalchemy import create_engine
# from sqlalchemy_utils import database_exists, create_database
import sqlalchemy as db
import json
import geojson
from geojson import Point, Feature


# Postgres Connection String
from config import User


## Connect to Postgres

In [9]:
# Create connection to the database
cestring = f'{User}GunData'
engine = create_engine(cestring)
metadata = db.MetaData()
connection = engine.connect()

# Save Table Connection Reference
incident = db.Table('incident', metadata, autoload=True, autoload_with=engine)

## Get Data from Table 

In [10]:
query = db.select([incident]) 
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

In [11]:
# move table to df
df = pd.DataFrame(ResultSet)
df.columns = ResultSet[0].keys()
df.dropna(subset=['Latitude', 'longitude'], inplace=True)

# Limit size for testing **
df_top =  df.head(1000)

# df_part = df_top.tail(250)
# df_part
# df

In [14]:
# Incidents by Date 
date = "2016-01-01"

df_date = df[df['Date'] == date]
df_date

,IncidentID,Date,State,City,No_Killed,Latitude,longitude,House_District,Senate_District
105723,477000,2016-01-01,Massachusetts,Boston,0,42.2802,-71.0759,NaN,NaN
105724,1023080,2016-01-01,Connecticut,Meriden,0,41.5402,-72.7895,84.0,13.0
105725,476977,2016-01-01,Missouri,Kansas City,0,39.0979,-94.5221,23.0,9.0
105726,476979,2016-01-01,Missouri,Kansas City,0,39.1649,-94.5833,18.0,17.0
105727,476661,2016-01-01,Wisconsin,Cudahy,2,42.9655,-87.8576,20.0,7.0
...,...,...,...,...,...,...,...,...,...
105939,477290,2016-01-01,Georgia,Armuchee,0,34.3856,-85.1782,12.0,52.0
105940,476589,2016-01-01,Illinois,Chicago,0,41.8737,-87.7352,10.0,5.0
105941,476580,2016-01-01,Kentucky,Hopkinsville,0,36.8621,-87.4950,NaN,NaN
105942,476550,2016-01-01,Illinois,Chicago,0,41.7575,-87.6248,34.0,17.0


In [21]:
year = 2017
# # Create a datetime series for later comparison
df['comp_date'] = pd.to_datetime(df['Date'], format= '%Y-%m-%d')
df_year = df[df['comp_date'].dt.year == year]
df_year

,IncidentID,Date,State,City,No_Killed,Latitude,longitude,House_District,Senate_District,comp_date
164474,740176,2017-01-01,Tennessee,Memphis,1,35.0670,-89.9156,91.0,33.0,2017-01-01
164475,741857,2017-01-01,Nebraska,Omaha,0,41.2960,-95.9470,NaN,11.0,2017-01-01
164476,741852,2017-01-01,Nebraska,Omaha,0,41.1918,-95.9528,NaN,5.0,2017-01-01
164477,740911,2017-01-01,Pennsylvania,Erie,0,42.1306,-80.0272,NaN,NaN,2017-01-01
164478,744584,2017-01-01,New Mexico,Portales,0,34.1898,-103.3240,63.0,27.0,2017-01-01
...,...,...,...,...,...,...,...,...,...,...
225872,1018014,2017-12-31,South Carolina,Florence,0,34.2093,-79.7614,59.0,30.0,2017-12-31
225873,1019711,2017-12-31,Florida,Cape Coral,0,26.5446,-82.0081,77.0,27.0,2017-12-31
225874,1020965,2017-12-31,Florida,Saint Petersburg,0,27.7730,-82.6330,68.0,19.0,2017-12-31
225875,1018101,2017-12-31,Illinois,Chicago,0,41.8553,-87.7069,NaN,NaN,2017-12-31


In [5]:
# df to json
# result = df_part.to_json(orient="records")
# parsed = json.loads(result)

# pretty json dump
# print(json.dumps(parsed, indent=4))

# inc_json = json.dumps(parsed, indent=4)
# print(parsed)

# result = dict(df_part)
# print(result)

In [6]:
# Generate return values geojson
geojson = {"type": "FeatureCollection", "features": []}

for _, row in df_top.iterrows():
    feature = {"type": "Feature", "geometry": {"type": "Point",
               "coordinates": [row['longitude'], row['Latitude']]},
               "properties": {"city": row['City'],"state": row['State'],
                             "incidentID": row['IncidentID'],"date": row['Date'],
                             "killed": row['No_Killed']}}
    geojson['features'].append(feature)
    
# return geojson
    
    
    
# print(json.dumps(geojson, indent=4))
# print('Done!')

In [7]:
# Write json file for testing
with open('Incident.json', 'w') as json_file:
    json.dump(geojson, json_file)  